In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi
from exact_sync.v1.api.teams_api import TeamsApi

from exact_sync.v1.models import ImageSet, Team, Product, AnnotationType, Image, Annotation, AnnotationMediaFile
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [4]:
configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = '****'
configuration.host =  "https://exact.cs.fau.de/"

image_set_ids = [251]

In [5]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [6]:
screened = [
    '10052_19 humane BAL Berliner Blau-001.svs',
    '10080_19 humane BAL Berliner Blau-001.svs',
    '10120_19 humane BAL Berliner Blau.svs',
    '10227_19 humane BAL Berliner Blau.svs',
    '10672_19 humane BAL Berliner Blau.svs',
    '10672_19 humane BAL Turnbull Blau.svs',
    '10835_19 humane BAL Berliner Blau.svs',
    '10835_19 humane BAL Turnbull Blau.svs',
    '11323_19 humane BAL Berliner Blau.svs',
    '11480_19 humane BAL Berliner Blau.svs',
    '2707_20 TB BAL Human neu.svs',
    '2702_20 BB Human BAL-001.svs'
]

In [7]:
images = {}
annotation_types = {}

for image_set in image_set_ids:
    for image in images_api.list_images(pagination=False, image_set=image_set).results:
        
        if image.name not in screened:
            continue
        
        images[image.id] = image
    for product in product_api.list_products(imagesets=image_set).results:
        for anno_type in annotation_types_api.list_annotation_types(product=product.id).results:
            annotation_types[anno_type.id] = anno_type

In [11]:
rows = []
for image_id, image in tqdm(images.items()):

    for annotation_type_id, annotation_type in annotation_types.items():

        for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, image=image_id, pagination=False, fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:
            
            species = "Human"
            data_set_name = "SDATA"
            version = "Inference"
            
            # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
            image_type = "Prussian" if "erliner" in image.name else "Turnbull"
            if  " TB " in image.name:
                image_type = "Turnbull"
            if " BB " in image.name:
                image_type = "Prussian"
                
            if anno.vector is not None and anno.user in [1, 11, 12]:
                rows.append([anno.id, image_id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

    df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
    df.to_pickle("EIPH_Human_SDATA_Inference_Annotations.pkl")

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [18:48<00:00, 94.00s/it]


In [12]:
rows = []
for image_id, image in tqdm(images.items()):

    for annotation_type_id, annotation_type in annotation_types.items():

        for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, 
                                                     image=image_id, pagination=False, deleted=False,
                                                     fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:
            
            species = "Human"
            data_set_name = "SDATA"
            version = "Screened"
            
            # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
            image_type = "Prussian" if "erliner" in image.name else "Turnbull"
            if  " TB " in image.name:
                image_type = "Turnbull"
            if " BB " in image.name:
                image_type = "Prussian"
            
            if anno.vector is not None and anno.user in [1, 11, 12]:
                rows.append([anno.id, image_id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

    df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
    df.to_pickle("EIPH_Human_SDATA_Screening_Annotations.pkl")

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [06:43<00:00, 33.61s/it]


In [8]:
rows = []
for image_set in [283]:
    for image in tqdm(images_api.list_images(pagination=False, image_set=image_set).results):

        for annotation_type_id, annotation_type in annotation_types.items():

            for anno in annotations_api.list_annotations(annotation_type=annotation_type_id, 
                                                         image=image.id, pagination=False, deleted=False,
                                                         fields="id,unique_identifier,annotation_type,vector,user,deleted,last_editor").results:

                species = "Human"
                data_set_name = "SDATA"
                version = "Density"

                # anno_id, image_id, image_name, anno_type_name, vector, user_id, user_name
                image_type = "Prussian" if "erliner" in image.name else "Turnbull"
                if  " TB " in image.name:
                    image_type = "Turnbull"
                if " BB " in image.name:
                    image_type = "Prussian"
                
                if anno.vector is not None and anno.user in [1, 11, 12]:
                    rows.append([anno.id, image.id, image.image_set, species, image.name, image_type, int(annotation_type.name), anno.vector, anno.unique_identifier, anno.user, anno.deleted, anno.last_editor, data_set_name, version])

        df = pd.DataFrame(rows, columns=['id', 'image_id', 'image_set', 'species' ,'image_name', 'image_type', 'grade', 'vector', 'unique_identifier', 'user_id', 'deleted', 'last_editor', 'data_set_name', 'version'])
        df.to_pickle("EIPH_Human_SDATA_Density_Annotations.pkl")

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [04:31<00:00, 22.59s/it]
